### This is My Capstone Project Notebook on Toronto Neighborhood

import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

now the list of postal codes from a wikipedia page

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

create a BeautifulSoup object to scrape a table from the page, and convert it to a dataframe

In [3]:
soup = BeautifulSoup(source.content, 'lxml')
table_html = soup.find('table', class_='wikitable sortable')
toronto_list = pd.read_html(str(table_html), header = 0)[0]
toronto_df = pd.DataFrame(toronto_list)

lets have a look at first few rows and shape of the dataframe

In [4]:
print(toronto_df.head())
print(toronto_df.shape)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(289, 3)


next, we'll remove the rows with "not assigned" boroughs

In [5]:
trnto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']

don't forget to reset the indices

In [6]:
trnto_df.reset_index(drop=True, inplace=True)
trnto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
trnto_df.shape

(212, 3)

we will use .groupby() to merge neighbourhoods of the same postalcode

In [8]:
grouped = trnto_df.groupby(['Postcode','Borough'],sort=False)['Neighbourhood'].apply(lambda x: ', '.join(x))
trnto_grouped = pd.DataFrame(grouped)
trnto_grouped.reset_index(inplace=True)
trnto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


next, find cells which have a borough but a Not assigned neighborhood, and assign the borough value to the neighbourhood

In [9]:
trnto_grouped['Neighbourhood'].replace('Not assigned',trnto_grouped['Borough'],inplace=True)

---

finally, let's look at our table and it's shape

In [10]:
trnto_grouped.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [11]:
trnto_grouped.shape

(103, 3)